In [1]:
import sys
import numpy as np
import pandas as pd
from scipy.stats import fisher_exact

%matplotlib inline
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import chi2_contingency

In [3]:
import mmrf_data_loader 

In [4]:
patient_list, patients_mask, event_duration, censorlist = mmrf_data_loader.load_MMRF_clinical_data()

659
10
659


In [5]:
clinical_data_df = pd.read_csv('data/km_clinical_data_669.csv')
clinical_data_df = clinical_data_df[clinical_data_df['subjectIDlist'].isin(patient_list)]
clinical_data_df = clinical_data_df.reset_index()

In [6]:
secondary_dataset_df = pd.read_excel('data/MMRF_clin_IA13_to_Anish.xlsx')
columns = ['sample', 'HRD', 't_MMSET', 't_CCND1', 'any_MAF',\
           't_MYC', 'chromothripsis_code', 'APOBEC_code',\
           'TPsum', 'gain1q21']
shorter_id_list = [] 
for id in patient_list: 
    shorter_id_list.append(id[0:14])

secondary_dataset_df = secondary_dataset_df[secondary_dataset_df['sample'].isin(shorter_id_list)]
secondary_dataset_df = secondary_dataset_df.reset_index()
secondary_dataset_df = secondary_dataset_df[columns]

In [7]:
secondary_dataset_df['TPsum'].value_counts()

0.0    418
1.0     53
2.0     18
Name: TPsum, dtype: int64

In [8]:
secondary_dataset_df.columns

Index(['sample', 'HRD', 't_MMSET', 't_CCND1', 'any_MAF', 't_MYC',
       'chromothripsis_code', 'APOBEC_code', 'TPsum', 'gain1q21'],
      dtype='object')

In [10]:
labels_df

,Unnamed: 0,0
0,0,1
1,1,1
2,2,2
3,3,1
4,4,1
...,...,...
437,437,2
438,438,1
439,439,2
440,440,2


In [12]:
input_label_fn = 'data/WEE1_labels3.csv'
labels_df = pd.read_csv(input_label_fn) 
labels_df['label'].value_counts()

3    224
1    218
0    217
Name: label, dtype: int64

In [13]:
input_label_fn = 'data/WEE1_labels3.csv'
output_table1_fn = 'table1_3class.csv'

low_key = 1
high_key = 3

labels_df = pd.read_csv(input_label_fn) 


table1 = pd.DataFrame({'Variable': ['Age', 'Sex', 'ISS', 'Treatment'],\
                       'Low WEE1':[0, 0, 0, 0], 'High WEE1':[0, 0, 0, 0],\
                       'p-val':[0, 0, 0, 0]}) 

low_val_ids = labels_df[labels_df['label']==low_key]['ID'].values
high_val_ids = labels_df[labels_df['label']==high_key]['ID'].values

low_age = clinical_data_df[clinical_data_df['subjectIDlist'].isin(low_val_ids)]['age_list'].values
high_age = clinical_data_df[clinical_data_df['subjectIDlist'].isin(high_val_ids)]['age_list'].values

table1.loc[0, 'Low WEE1'] = np.round(np.mean(low_age), 4) 
table1.loc[0, 'High WEE1'] = np.round(np.mean(high_age), 4) 
table1.loc[0, 'p-val'] = np.round(stats.ttest_ind(low_age, high_age).pvalue, 4) 

low_series = clinical_data_df[clinical_data_df['subjectIDlist'].isin(low_val_ids)]['gender_list'].value_counts()
high_series = clinical_data_df[clinical_data_df['subjectIDlist'].isin(high_val_ids)]['gender_list'].value_counts()
table = (low_series[1], low_series[2]), (high_series[1], high_series[2])
oddsr, p  = stats.fisher_exact(table)   
low_str = 'M: ' + str(low_series[1]) +'; '+ 'F: ' + str(low_series[2])
high_str = 'M: ' + str(high_series[1]) +'; '+ 'F: ' + str(high_series[2])

table1.loc[1, 'Low WEE1'] = low_str
table1.loc[1, 'High WEE1'] = high_str
table1.loc[1, 'p-val'] = p

low_series = clinical_data_df[clinical_data_df['subjectIDlist'].isin(low_val_ids)]['iss_list'].value_counts()
high_series = clinical_data_df[clinical_data_df['subjectIDlist'].isin(high_val_ids)]['iss_list'].value_counts()
table = [[low_series[1], low_series[2], low_series[3]], [high_series[1], high_series[2], high_series[3]]]
res = stats.chi2_contingency(table)   
low_str = '(1): ' + str(low_series[1]) +'; '+ '(2): ' + str(low_series[2]) + '; (3): ' + str(low_series[3])
high_str = '(1): ' + str(high_series[1]) +'; '+ '(2): ' + str(high_series[2]) + '; (3): ' + str(high_series[3])

table1.loc[2, 'Low WEE1'] = low_str
table1.loc[2, 'High WEE1'] = high_str
table1.loc[2, 'p-val'] = res.pvalue

low_therapy_series = clinical_data_df[clinical_data_df['subjectIDlist'].isin(low_val_ids)]['therapy_list'].value_counts()
low_output_str = ""
for n in range(0, len(low_therapy_series.index)): 
    low_output_str = low_output_str + low_therapy_series.index[n] + ": " + str(low_therapy_series.values[n]) + "; "

high_therapy_series = clinical_data_df[clinical_data_df['subjectIDlist'].isin(high_val_ids)]['therapy_list'].value_counts()
high_output_str = ""
for n in range(0, len(high_therapy_series.index)): 
    high_output_str = high_output_str + high_therapy_series.index[n] + ": " + str(high_therapy_series.values[n]) + "; "

table1.loc[3, 'Low WEE1'] = low_output_str
table1.loc[3, 'High WEE1'] = high_output_str
table1.loc[3, 'p-val'] = "-"

# table1.to_csv(output_table1_fn)

In [14]:
table1

,Variable,Low WEE1,High WEE1,p-val
0,Age,63.4312,61.1027,0.0212
1,Sex,M: 114; F: 104,M: 139; F: 85,0.043518
2,ISS,(1): 75; (2): 88; (3): 48,(1): 79; (2): 73; (3): 69,0.080385
3,Treatment,combined bortezomib/IMIDs-based: 124; Bortezom...,combined bortezomib/IMIDs-based: 89; combined ...,-


In [15]:
table1 = table1.set_index('Variable') 

In [16]:
table1

,Low WEE1,High WEE1,p-val
Variable,,,
Age,63.4312,61.1027,0.0212
Sex,M: 114; F: 104,M: 139; F: 85,0.043518
ISS,(1): 75; (2): 88; (3): 48,(1): 79; (2): 73; (3): 69,0.080385
Treatment,combined bortezomib/IMIDs-based: 124; Bortezom...,combined bortezomib/IMIDs-based: 89; combined ...,-


In [19]:
def pad_vector(low_feature):  
    # print("pad") 
    # print(low_feature) 
    if len(low_feature.index) < 3: 
        if low_feature.index[0] == 0: 
            if low_feature.index[1] == 1: 
                low_feature.loc[2.0] = 0 
    return low_feature

In [20]:
input_label_fn = 'data/WEE1_labels3.csv'
output_table2_fn = 'table1_3class.csv'
low_key = 1
high_key = 3



rows = ['HRD', 't_MMSET', 't_CCND1', 'any_MAF',\
           't_MYC', 'chromothripsis_code', 'APOBEC_code']
columns = ['Low WEE1', 'High WEE1', 'p-val']
table2 = pd.DataFrame(np.nan, index=rows, columns=columns)

labels_df = pd.read_csv(input_label_fn) 

low_val_ids = labels_df[labels_df['label']==low_key]['ID'].values
high_val_ids = labels_df[labels_df['label']==high_key]['ID'].values

for n in range(0, len(low_val_ids)): 
    low_val_ids[n] = low_val_ids[n][0:14]

for n in range(0, len(high_val_ids)): 
    high_val_ids[n] = high_val_ids[n][0:14]

for feature_name in rows: 
    low_feature = secondary_dataset_df[secondary_dataset_df['sample'].isin(low_val_ids)][feature_name].dropna().value_counts()
    high_feature = secondary_dataset_df[secondary_dataset_df['sample'].isin(high_val_ids)][feature_name].dropna().value_counts()
    
    table = (low_feature[0], low_feature[1]), (high_feature[0], high_feature[1])
    oddsr, p  = stats.fisher_exact(table)   
    low_str = '(0): ' + str(low_feature[0]) +'; '+ '(1): ' + str(low_feature[1])
    high_str = '(0): ' + str(high_feature[0]) +'; '+ '(1): ' + str(high_feature[1])
    
    table2.loc[feature_name, 'Low WEE1'] = low_str
    table2.loc[feature_name, 'High WEE1'] = high_str
    table2.loc[feature_name, 'p-val'] = "{:.4e}".format(p) 

three_level_features = ['gain1q21', 'TPsum']

for feature_name in three_level_features: 
    low_feature = secondary_dataset_df[secondary_dataset_df['sample'].isin(low_val_ids)][feature_name].dropna().value_counts()
    high_feature = secondary_dataset_df[secondary_dataset_df['sample'].isin(high_val_ids)][feature_name].dropna().value_counts()

    low_feature = pad_vector(low_feature) 
    
    table = [[low_feature[0], low_feature[1], low_feature[2]], [high_feature[0], high_feature[1], high_feature[2]]]
    res = stats.chi2_contingency(table)   
    low_str = '(0): ' + str(low_feature[0]) +'; '+ '(1): ' + str(low_feature[1]) + '; (2): ' + str(low_feature[2])
    high_str = '(0): ' + str(high_feature[0]) +'; '+ '(1): ' + str(high_feature[1]) + '; (2): ' + str(high_feature[2])
    
    table2.loc[feature_name, 'Low WEE1'] = low_str
    table2.loc[feature_name, 'High WEE1'] = high_str
    table2.loc[feature_name, 'p-val'] = "{:.4e}".format(res.pvalue)
    
# table2.to_csv(output_table2_fn)






In [21]:
table2

,Low WEE1,High WEE1,p-val
HRD,(0): 38; (1): 151,(0): 112; (1): 58,7.9401e-19
t_MMSET,(0): 182; (1): 22,(0): 161; (1): 20,1.0000e+00
t_CCND1,(0): 193; (1): 11,(0): 112; (1): 69,7.1623e-16
any_MAF,(0): 200; (1): 4,(0): 159; (1): 22,6.1587e-05
t_MYC,(0): 165; (1): 39,(0): 162; (1): 19,2.2047e-02
chromothripsis_code,(0): 162; (1): 42,(0): 125; (1): 56,2.5686e-02
APOBEC_code,(0): 198; (1): 6,(0): 154; (1): 25,1.0211e-04
gain1q21,(0): 141; (1): 47; (2): 1,(0): 105; (1): 46; (2): 19,3.4814e-05
TPsum,(0): 157; (1): 15; (2): 0,(0): 114; (1): 22; (2): 13,5.5456e-05


In [22]:
low_feature.index.values

array([0., 1., 2.])

In [23]:
table2

,Low WEE1,High WEE1,p-val
HRD,(0): 38; (1): 151,(0): 112; (1): 58,7.9401e-19
t_MMSET,(0): 182; (1): 22,(0): 161; (1): 20,1.0000e+00
t_CCND1,(0): 193; (1): 11,(0): 112; (1): 69,7.1623e-16
any_MAF,(0): 200; (1): 4,(0): 159; (1): 22,6.1587e-05
t_MYC,(0): 165; (1): 39,(0): 162; (1): 19,2.2047e-02
chromothripsis_code,(0): 162; (1): 42,(0): 125; (1): 56,2.5686e-02
APOBEC_code,(0): 198; (1): 6,(0): 154; (1): 25,1.0211e-04
gain1q21,(0): 141; (1): 47; (2): 1,(0): 105; (1): 46; (2): 19,3.4814e-05
TPsum,(0): 157; (1): 15; (2): 0,(0): 114; (1): 22; (2): 13,5.5456e-05


In [24]:
merged_table = pd.concat([table1, table2], axis=0)

In [25]:
merged_table = merged_table.rename(index={'t_MMSET': 't(4;14)', 't_CCND1': 't(11;14)', 'any_MAF':'MAF translocation', \
                 't_MYC':'MYC translocation', 'chromothripsis_code': 'Chromothripsis',\
                 'APOBEC_code': 'Hyper APOBEC', 'TPsum':'TP53 mutation'})


In [26]:
merged_table

,Low WEE1,High WEE1,p-val
Age,63.4312,61.1027,0.0212
Sex,M: 114; F: 104,M: 139; F: 85,0.043518
ISS,(1): 75; (2): 88; (3): 48,(1): 79; (2): 73; (3): 69,0.080385
Treatment,combined bortezomib/IMIDs-based: 124; Bortezom...,combined bortezomib/IMIDs-based: 89; combined ...,-
HRD,(0): 38; (1): 151,(0): 112; (1): 58,7.9401e-19
t(4;14),(0): 182; (1): 22,(0): 161; (1): 20,1.0000e+00
t(11;14),(0): 193; (1): 11,(0): 112; (1): 69,7.1623e-16
MAF translocation,(0): 200; (1): 4,(0): 159; (1): 22,6.1587e-05
MYC translocation,(0): 165; (1): 39,(0): 162; (1): 19,2.2047e-02
Chromothripsis,(0): 162; (1): 42,(0): 125; (1): 56,2.5686e-02


In [31]:
merged_table = merged_table.drop("Treatment", axis='index')

array(['Age', 'Sex', 'ISS', 'HRD', 't(4;14)', 't(11;14)',
       'MAF translocation', 'MYC translocation', 'Chromothripsis',
       'Hyper APOBEC', 'gain1q21', 'TP53 mutation'], dtype=object)

In [33]:
from statsmodels.stats.multitest import multipletests

In [39]:
foo = merged_table.index.to_numpy()

In [42]:
pvalues = merged_table['p-val'].values

In [46]:
pvalues = pvalues.astype(np.float64) 

In [48]:
reject, pvals_corrected, a, b = multipletests(pvalues, alpha=0.05, method='fdr_bh')    
# foo2 = foo[reject]

In [50]:
foo

array(['Age', 'Sex', 'ISS', 'HRD', 't(4;14)', 't(11;14)',
       'MAF translocation', 'MYC translocation', 'Chromothripsis',
       'Hyper APOBEC', 'gain1q21', 'TP53 mutation'], dtype=object)

In [51]:
pvals_corrected

array([3.30705000e-02, 5.22221234e-02, 8.76924133e-02, 9.52812000e-18,
       1.00000000e+00, 4.29738000e-15, 1.47808800e-04, 3.30705000e-02,
       3.42480000e-02, 2.04220000e-04, 1.39256000e-04, 1.47808800e-04])

In [49]:
foo[reject]

array(['Age', 'HRD', 't(11;14)', 'MAF translocation', 'MYC translocation',
       'Chromothripsis', 'Hyper APOBEC', 'gain1q21', 'TP53 mutation'],
      dtype=object)

In [53]:
pd.DataFrame({'Index':foo, 'pvals_fdr': pvals_corrected})

,Index,pvals_fdr
0,Age,3.307050e-02
1,Sex,5.222212e-02
2,ISS,8.769241e-02
3,HRD,9.528120e-18
4,t(4;14),1.000000e+00
5,t(11;14),4.297380e-15
6,MAF translocation,1.478088e-04
7,MYC translocation,3.307050e-02
8,Chromothripsis,3.424800e-02
9,Hyper APOBEC,2.042200e-04


In [51]:
merged_table.to_csv('table1_3.csv') 